In [1]:
# Global variables for condition 
global participant
participant = '050'

# Global variables for condition 
global condition
condition = 'Ball'  #'Ball' or 'NB'

#and hand type
global hand_type
hand_type = 'LH' #BH RH LH


# Configuration cell for video paths
global video_paths
video_paths = {
    #"Videopanel_Sync": f"/Users/beorn/Dropbox/0WORK/3Project Wensen/EMG Kin Research Experiment/Data/Experiment Data/SDC_{participant}/Videos/Videopanel/SDC_{participant}_{condition}_{hand_type}_VP.MP4"
    # "Videopanel_Sync": f"/Volumes/Samsung_T5/Experiment Data/SDC_{participant}/Videos/Videopanel/SDC_{participant}_{condition}_{hand_type}_VP.MP4"
    "Videopanel_Sync": f"/Volumes/Beorn_4T/Experiment Data/SDC_{participant}/Videos/Videopanel/SDC_{participant}_{condition}_{hand_type}_VP.MP4"
    # "Videopanel_Sync": f"D:\Experiment Data\SDC_{participant}\Videos\Videopanel\SDC_{participant}_{condition}_{hand_type}_VP.MP4"

}

# Configuration cell for video paths
global file_paths
file_paths = {
    # "File_Sync": f"/Users/beorn/Dropbox/0WORK/3Project Wensen/EMG Kin Research Experiment/Data/Experiment Data/SDC_{participant}/EMG/"
    "File_Sync": f"/Volumes/Beorn_4T/Experiment Data/SDC_{participant}/EMG/"
    # "File_Sync": f"D:\Experiment Data\SDC_{participant}\EMG"
}

# Configuration cell for video paths
global write_paths
write_paths = {
    #"Write_Sync": f"/Users/beorn/Dropbox/0WORK/3Project Wensen/EMG Kin Research Experiment/Data/Experiment Data/SDC_{participant}/Videos/Videopanel/TESTING_Frame_Sync.MP4"
    "Write_Sync": f"/Volumes/Beorn_4T/Experiment Data/SDC_{participant}/Videos/Videopanel/TESTING_Frame_Sync.MP4"
    # "Write_Sync": f"D:\Experiment Data\SDC_{participant}\Videos/Videopanel\TESTING_Frame_Sync.MP4"
}

# Configuration cell for video paths
global save_paths
save_paths = {
    # "Save_Sync": f"/Users/beorn/Dropbox/0WORK/3Project Wensen/EMG Kin Research Experiment/Data/Experiment Data/SDC_{participant}/aligned_data.mat"
    "Save_Sync": f"/Volumes/Beorn_4T/Experiment Data/SDC_{participant}/aligned_data.mat"
    # "Save_Sync": f"D:\Experiment Data\SDC_{participant}\aligned_data.mat"    
    
}

# Configuration cell for video paths
global aligned_dat_save
aligned_data_save = {
    # "aligned_data_path": f"/Users/beorn/Dropbox/0WORK/3Project Wensen/EMG Kin Research Experiment/Data/Experiment Data/SDC_{participant}/Time_Axis_Alignmen_Python.json"
    "aligned_data_path": f"/Volumes/Beorn_4T/Experiment Data/SDC_{participant}/Time_Axis_Alignmen_Python.json"
    # "aligned_data_path": f"D:\Experiment Data\SDC_{participant}\Time_Axis_Alignmen_Python.json"

}

In [ ]:
# creating start time for GoPro Video and producin ga time vector based on that

%matplotlib widget

import subprocess
import pandas as pd
from datetime import datetime, timezone
import numpy as np



# Define the paths to your video files using the video_paths dictionary
video_file_path = video_paths["Videopanel_Sync"]  # Ensure the key matches exactly what you defined above


# # Define the path to your video file
# video_file_path = "/Users/beorn/Dropbox/0WORK/3Project Wensen/EMG Kin Research Experiment/Data/Experiment Data/SDC_007/Videos/Videopanel/SDC_007_Ball_RH_VP_synced_time_injection.MP4"


# Construct the ffprobe command to extract specific metadata
metadata_command = (
    f'ffprobe -v error '
    f'-show_entries stream_tags=timecode '
    f'-show_entries format_tags=creation_time '
    f'-select_streams v:0 '  # Focus on video stream for timecode; adjust if needed
    f'-show_entries stream=nb_frames,r_frame_rate,duration '
    f'-of default=noprint_wrappers=1 "{video_file_path}"'
)

# Execute the command and capture the output
metadata_process = subprocess.run(metadata_command, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if metadata_process.returncode == 0:
    print("Metadata extracted successfully.")
    
    # Initialize a dictionary to hold the metadata
    data = {'Total Frames': None, 'Timecode': None, 'Creation Time': None, 'Frame Rate': None}
    
    # Parse the output
    for line in metadata_process.stdout.split('\n'):
        if 'nb_frames' in line:
            key, value = line.split('=')
            data['Total Frames'] = value.strip() if value.strip().isdigit() else 'Calculation needed'
        elif 'TAG:creation_time' in line:
            _, value = line.split('=')
            data['Creation Time'] = value.strip()
        elif 'TAG:timecode' in line:
            _, value = line.split('=')
            data['Timecode'] = value.strip()
        elif 'r_frame_rate' in line:
            _, value = line.split('=')
            # Simplify frame rate to a single number if possible
            numerator, denominator = map(int, value.strip().split('/'))
            data['Frame Rate'] = str(numerator / denominator if denominator != 0 else numerator)
    
    # Convert the dictionary into a DataFrame for nicer display
    df = pd.DataFrame(list(data.items()), columns=['Tag', 'Value'])
    
    # Display the DataFrame
    print(df)
else:
    print("Error extracting metadata:", metadata_process.stderr)


    
# Assuming 'df' is your DataFrame
creation_time_str = df.loc[df['Tag'] == 'Creation Time', 'Value'].iloc[0]
timecode_str = df.loc[df['Tag'] == 'Timecode', 'Value'].iloc[0]
frame_rate = df.loc[df['Tag'] == 'Frame Rate', 'Value'].iloc[0]  # Directly using frame rate value

# Splitting creation time to get the date and hour part
creation_date_hour = creation_time_str.split('T')[0]
creation_hour = creation_time_str.split('T')[1][:2]

# Extracting minutes, seconds, and hundredths from the timecode
timecode_minutes = timecode_str.split(':')[1]
timecode_seconds = timecode_str.split(':')[2]
timecode_hundredths = timecode_str.split(':')[3]

# Combine them without altering the frame rate usage
combined_timecode = f"{creation_date_hour}T{creation_hour}:{timecode_minutes}:{timecode_seconds}.{timecode_hundredths}Z"

print("Combined Timecode:", combined_timecode)
print("Frame Rate:", frame_rate)


# Assuming combined_timecode is your final timecode from previous steps
combined_timecode_str = f"{creation_date_hour}T{creation_hour}:{timecode_minutes}:{timecode_seconds}.{timecode_hundredths}Z"

# Parse the combined timecode into a datetime object
combined_timecode_datetime = datetime.strptime(combined_timecode_str, "%Y-%m-%dT%H:%M:%S.%fZ")

# Convert the datetime object to UNIX timestamp
unix_timestamp = combined_timecode_datetime.replace(tzinfo=timezone.utc).timestamp()

print("UNIX Timestamp:", unix_timestamp)





# Assuming combined_timecode_datetime is already defined
unix_timestamp_microseconds = int(combined_timecode_datetime.replace(tzinfo=timezone.utc).timestamp() * 1e6)

print("UNIX Timestamp (microseconds):", unix_timestamp_microseconds)


# Assuming df is your existing DataFrame and unix_timestamp_microseconds is defined
new_row = pd.DataFrame({'Tag': ['UNIX Time'], 'Value': [str(unix_timestamp_microseconds)]})

# Using pandas.concat to add the new row
df = pd.concat([df, new_row], ignore_index=True)

print(df)



# Extract values from DataFrame
unix_time_start = int(df.loc[df['Tag'] == 'UNIX Time', 'Value'].iloc[0])
frame_rate = float(df.loc[df['Tag'] == 'Frame Rate', 'Value'].iloc[0])
total_frames = int(df.loc[df['Tag'] == 'Total Frames', 'Value'].iloc[0])

# Calculate time increment per frame in microseconds
time_increment_per_frame = int((1 / frame_rate) * 1e6)

# Generate the time vector
time_vector = np.array([unix_time_start + i * time_increment_per_frame for i in range(total_frames)])

# Display the first few elements of the time vector and its total length
print("First few UNIX Times in Vector:", time_vector[:5])
print("Length of Time Vector:", len(time_vector))


In [ ]:
#loading the EMG and Kinematics
import pandas as pd
import os
import numpy as np


file_location = file_paths["File_Sync"]  # Ensure the key matches exactly what you defined above

# Base file location and code name
# file_location = '/Users/beorn/Dropbox/0WORK/3Project Wensen/EMG Kin Research Experiment/Data/Experiment Data/SDC_007/EMG/2024-04-08_SDC_007_SD_Session1'
codename = participant

# Sensor names and file patterns
RawSensorData = [
    ('MagSensor_1', 'SDC_%s_Session1_Shimmer_7ED1_Calibrated_SD.dat'),
    ('MagSensor_2', 'SDC_%s_Session1_Shimmer_7EDB_Calibrated_SD.dat'),
    ('Right_Arm', 'SDC_%s_Session1_Shimmer_8AAA_Calibrated_SD.dat'),
    ('Left_Arm', 'SDC_%s_Session1_Shimmer_8009_Calibrated_SD.dat'),
    ('Supinator_Flex_Carp_Uln_R', 'SDC_%s_Session1_Shimmer_88F1_Calibrated_SD.dat'),
    ('Pron_Teres_pron_quad_R', 'SDC_%s_Session1_Shimmer_88F4_Calibrated_SD.dat'),
    ('Supinator_Flex_Carp_Uln_L', 'SDC_%s_Session1_Shimmer_88F6_Calibrated_SD.dat'),
    ('Pron_Teres_pron_quad_L', 'SDC_%s_Session1_Shimmer_8FCF_Calibrated_SD.dat'),
    ('Putter_Sensor', 'SDC_%s_Session1_Shimmer_541A_Calibrated_SD.dat'),
]


# for if the files happen to be CSV
# RawSensorData = [
#     ('MagSensor_1', 'SDC_%s_Session1_Shimmer_7ED1_Calibrated_SD.csv'),
#     ('MagSensor_2', 'SDC_%s_Session1_Shimmer_7EDB_Calibrated_SD.csv'),
#     ('Right_Arm', 'SDC_%s_Session1_Shimmer_8AAA_Calibrated_SD.csv'),
#     ('Left_Arm', 'SDC_%s_Session1_Shimmer_8009_Calibrated_SD.csv'),
#     ('Supinator_Flex_Carp_Uln_R', 'SDC_%s_Session1_Shimmer_88F1_Calibrated_SD.csv'),
#     ('Pron_Teres_pron_quad_R', 'SDC_%s_Session1_Shimmer_88F4_Calibrated_SD.csv'),
#     ('Supinator_Flex_Carp_Uln_L', 'SDC_%s_Session1_Shimmer_88F6_Calibrated_SD.csv'),
#     ('Pron_Teres_pron_quad_L', 'SDC_%s_Session1_Shimmer_8FCF_Calibrated_SD.csv'),
#     ('Putter_Sensor', 'SDC_%s_Session1_Shimmer_541A_Calibrated_SD.csv'),
# ]

# Initialize a list to keep track of the sensor names for which DataFrames are created
loaded_sensors = []

# Process each file based on the RawSensorData list
for sensor_name, file_pattern in RawSensorData:
    formatted_filename = file_pattern % codename
    file_path = os.path.join(file_location, formatted_filename)
    
    # Attempt to read the file into a DataFrame and assign it to a global variable
    try:
        df = pd.read_csv(file_path, delimiter='\t', header=0, skiprows=[0, 1], dtype=float)
        
        # Adjust the 'ms' column to the desired format
        if 'ms' in df.columns:
            df['ms'] = (df['ms'] * 1e3).astype(np.int64)
        
        globals()[sensor_name] = df
        loaded_sensors.append(sensor_name)  # Keep track of the loaded sensor names
        print(f"{sensor_name} loaded successfully.")
    except Exception as e:
        print(f"Failed to load {sensor_name} due to: {e}")

# Now, `loaded_sensors` contains the names of all sensors for which DataFrames have been successfully loaded
# You can access any DataFrame using globals(), e.g., globals()['MagSensor_1']

# Cleanup
del codename, file_location, file_path, file_pattern, formatted_filename, sensor_name


In [ ]:
# check for and replace corrupted time axis data
import pandas as pd
import numpy as np
import os

def check_and_fix_time(df):
    """
    Function to fix repeated time points in the DataFrame by creating synthetic time points.
    """
    total_observations = df.shape[0]

    # Find the first repeating time point
    unique_times, counts = np.unique(df.iloc[:, 0], return_counts=True)
    first_repeating_index = np.where(counts > 1)[0]

    if len(first_repeating_index) == 0:
        print('No repeating time points found.')
        return df
    else:
        print('One of the time axes has duplicates.')
        first_repeating_time = unique_times[first_repeating_index[0]]

        # Index of the first repeating time in the original DataFrame
        start_index = df.index[df.iloc[:, 0] == first_repeating_time][0]

        # Calculate average time interval based on the data before the first repeat
        time_intervals = np.diff(df.iloc[:start_index, 0])
        average_interval = np.mean(time_intervals)

        # Number of synthetic points to generate
        synthetic_points = total_observations - start_index

        # Generate synthetic time points
        start_time = df.iloc[start_index - 1, 0]
        synthetic_times = start_time + (np.arange(1, synthetic_points + 1) * average_interval)

        # Replace the repeating entries in the original DataFrame
        df.iloc[start_index:, 0] = synthetic_times

        return df

# Now integrate this function with your existing code
loaded_sensors = []

for sensor_name, file_pattern in RawSensorData:
    formatted_filename = file_pattern % participant
    file_path = os.path.join(file_paths["File_Sync"], formatted_filename)
    
    try:
        df = pd.read_csv(file_path, delimiter='\t', header=0, skiprows=[0, 1], dtype=float)
        
        if 'ms' in df.columns:
            df['ms'] = (df['ms'] * 1e3).astype(np.int64)
        
        # Fix time axis if necessary
        df = check_and_fix_time(df)
        
        globals()[sensor_name] = df
        loaded_sensors.append(sensor_name)
        print(f"{sensor_name} loaded successfully.")
    except Exception as e:
        print(f"Failed to load {sensor_name} due to: {e}")




In [ ]:
#Interpolating sensors to have the same time axis

# Assuming the reference DataFrame for 'MagSensor_1' is already loaded and defined
reference_df = globals()['MagSensor_1']
reference_time_axis = reference_df.iloc[:, 0]  # Assuming the first column is UNIX time

# Loop through each sensor DataFrame, interpolate it against the reference time axis, and update the global namespace
for sensor_name, _ in RawSensorData:
    if sensor_name == 'MagSensor_1':
        # Optionally, you could also store the reference DataFrame with an 'interpolated' prefix if needed
        globals()['interpolated_' + sensor_name] = reference_df
        continue  # Skip interpolation for the reference sensor itself
    
    # Retrieve the current DataFrame
    current_df = globals()[sensor_name]
    
    # Assuming the first column of each DataFrame is UNIX time, align and interpolate
    current_df_indexed = current_df.set_index(current_df.columns[0])
    aligned_df = current_df_indexed.reindex(reference_time_axis, method='nearest').interpolate(method='linear')
    aligned_df.reset_index(inplace=True)
    
    # Update the global namespace with the interpolated DataFrame
    interpolated_name = 'interpolated_' + sensor_name
    globals()[interpolated_name] = aligned_df

    # Optionally print a success message
    print(f"{interpolated_name} created and stored successfully.")



In [ ]:
#create a container to store data retrieve the unix kinematic and video time vector
import pandas as pd

class DataFrameContainer:
    def __init__(self):
        self.frames = {}
    
    def add_frame(self, name, dataframe):
        """Add a DataFrame to the container."""
        self.frames[name] = dataframe
    
    def get_frame(self, name):
        """Retrieve a DataFrame by name."""
        return self.frames.get(name, None)
    
    def get_all_frames(self):
        """Retrieve all stored DataFrames."""
        return self.frames
    
    def get_frame_names(self):
        """Retrieve the names of all stored DataFrames."""
        return list(self.frames.keys())

# Create an instance of the container
Sensor_Container = DataFrameContainer()

# Assuming you have DataFrames named like 'interpolated_MagSensor_1', 'interpolated_MagSensor_2', etc.
# Loop through the global namespace to find and add them to the container
for var_name, var_value in list(globals().items()):
    if isinstance(var_value, pd.DataFrame) and var_name.startswith('interpolated_'):
        Sensor_Container.add_frame(var_name, var_value)

# Assuming time_vector is an array-like structure with UNIX timestamps
time_vector_df = pd.DataFrame(time_vector, columns=['UNIX Time'])
# Adding the time_vector_df to the Sensor_Container
Sensor_Container.add_frame('time_vector', time_vector_df)

# Now, you can retrieve the names of all stored DataFrames using the new method
frame_names = Sensor_Container.get_frame_names()
print("Stored DataFrame names:", frame_names)

# Assuming 'Sensor_Container' is an object storing all your DataFrames, including 'time_vector' and 'interpolated_MagSensor_1'
# Retrieve the 'UNIX Time' column from 'time_vector' DataFrame
video_time_vector_unix = Sensor_Container.get_frame('time_vector')['UNIX Time'].values
# Retrieve the 'ms' column from 'interpolated_MagSensor_1' DataFrame
kinematics_time_vector_unix = Sensor_Container.get_frame('interpolated_Right_Arm')['ms'].values


In [ ]:
# creat a dataframe dictionary - dict - to retrieve desired vectors
# List of DataFrame names you're interested in
dataframe_names = [
    'interpolated_MagSensor_1', 'interpolated_MagSensor_2', 'interpolated_Right_Arm',
    'interpolated_Left_Arm', 'interpolated_Supinator_Flex_Carp_Uln_R',
    'interpolated_Pron_Teres_pron_quad_R', 'interpolated_Supinator_Flex_Carp_Uln_L',
    'interpolated_Pron_Teres_pron_quad_L', 'interpolated_Putter_Sensor'
]

# Initialize an empty dictionary to store the DataFrames
Sensor_Dict = {}

# Iterate over the list of DataFrame names
for df_name in dataframe_names:
    # Assuming each DataFrame name corresponds to a variable in the global namespace
    # The globals() function is used to access these variables dynamically
    Sensor_Dict[df_name] = globals()[df_name]

# At this point, dataframes_dict contains all the specified DataFrames


In [ ]:
#create a bandstop filter for the acceleration signals in the right arm 
import numpy as np
from scipy.signal import butter, sosfilt
import matplotlib.pyplot as plt

# Assuming Sensor_Dict is your dictionary with sensor data
# Define sensor names and columns of interest
sensor_names = ['interpolated_Right_Arm', 'interpolated_Left_Arm']
columns = ['m/(s^2)', 'm/(s^2).1', 'm/(s^2).2']

# Sampling rate and filter parameters
fs = 512  # Sampling rate in Hz

# Design the bandstop filters
sos_1 = butter(N=4, Wn=[20, 85], btype='bandstop', fs=fs, output='sos')
sos_2 = butter(N=4, Wn=[100, 240], btype='bandstop', fs=fs, output='sos')

# Apply the filters to the data
for sensor in sensor_names:
    if sensor in Sensor_Dict:
        for column in columns:
            # Apply filters sequentially directly on Sensor_Dict
            Sensor_Dict[sensor][column] = sosfilt(sos_1, Sensor_Dict[sensor][column])
            Sensor_Dict[sensor][column] = sosfilt(sos_2, Sensor_Dict[sensor][column])

# Example plot for one sensor and column
sensor_example = 'interpolated_Right_Arm'
column_example = 'm/(s^2)'

# # ______HERE IS FOR PLOTTING TO SEE HOW WE ARE DOING________
# plt.figure(figsize=(10, 6))
# # Note: Since Sensor_Dict is now altered, plotting "Original" would actually plot the filtered data
# # To plot original vs. filtered comparison, you would need to save the original data before filtering
# plt.plot(Sensor_Dict[sensor_example].index, Sensor_Dict[sensor_example][column_example], label='Filtered', alpha=0.5, linestyle='--')
# plt.legend()
# plt.title(f"{sensor_example} - {column_example}: Filtered")
# plt.xlabel('Sample Index')
# plt.ylabel('Acceleration (m/s^2)')
# plt.show()


In [ ]:
#match unix time stamps from video with the corresponding closest values from the kineamtics
import numpy as np

def find_closest_indices(main_array, reference_array):
    """
    For each timestamp in the main_array, find the index of the closest timestamp in the reference_array.
    """
    indices = np.zeros(len(main_array), dtype=np.int64)
    for i, value in enumerate(main_array):
        indices[i] = np.argmin(np.abs(reference_array - value))
    return indices

def synchronize_datasets(video_time_vector_unix, kinematics_time_vector_unix):
    # Identify the overlapping period
    overlap_start = max(video_time_vector_unix[0], kinematics_time_vector_unix[0])
    overlap_end = min(video_time_vector_unix[-1], kinematics_time_vector_unix[-1])
    
    # Filter the kinematics timestamps to the overlapping period
    kinematics_indices = np.where((kinematics_time_vector_unix >= overlap_start) & (kinematics_time_vector_unix <= overlap_end))[0]
    kinematics_overlap_timestamps = kinematics_time_vector_unix[kinematics_indices]
    
    # Filter the video timestamps to the overlapping period
    video_indices = np.where((video_time_vector_unix >= overlap_start) & (video_time_vector_unix <= overlap_end))[0]
    video_overlap_timestamps = video_time_vector_unix[video_indices]
    
    # Find the closest kinematics timestamps to each video timestamp
    closest_kinematics_indices = find_closest_indices(video_overlap_timestamps, kinematics_overlap_timestamps)
    
    # Prepare the output array with additional columns
    # Column 0: Aligned index in kinematics_time_vector_unix
    # Column 1: Corresponding kinematics timestamp
    # Column 2: Aligned index in video_time_vector_unix (for reference)
    # Column 3: Corresponding video timestamp
    aligned_data = np.zeros((len(video_indices), 3), dtype=np.int64)
    aligned_data[:, 0] = kinematics_indices[closest_kinematics_indices]
    aligned_data[:, 1] = kinematics_time_vector_unix[aligned_data[:, 0]]
    aligned_data[:, 2] = video_time_vector_unix[video_indices]
    
    return aligned_data

# Assuming your time vectors are named kinematics_time_vector_unix and video_time_vector_unix
aligned_data = synchronize_datasets(video_time_vector_unix, kinematics_time_vector_unix)

# The aligned_data array now contains the aligned indices and their corresponding timestamps


In [ ]:
# Initialize an empty dictionary to store the synced selected kinematic and EMG values camera_sycned_sensors
Camera_Synced_Sensors = {}

# Extract the indices from the first column of aligned_data
indices = aligned_data[:, 0]

# Iterate over each item in Sensor_Dict
for name, df in Sensor_Dict.items():
    # Check if the name starts with 'interpolated_'
    if name.startswith('interpolated_'):
        # Select rows from the DataFrame based on the extracted indices
        # Note: Ensure that the indices are valid for the DataFrame
        filtered_df = df.iloc[indices]
        
        # Store the filtered DataFrame in the new dictionary
        Camera_Synced_Sensors[name] = filtered_df

# At this point, 'filtered_dataframes' contains all the filtered DataFrames
# from Sensor_Dict that start with 'interpolated_', with rows corresponding to the indices in aligned_data


In [ ]:
#plot decimated kin/emg 
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Assuming Camera_Synced_Sensors is a dictionary with your DataFrames

# Access the DataFrames directly from the Camera_Synced_Sensors dictionary
interpolated_MagSensor_1 = Camera_Synced_Sensors['interpolated_MagSensor_1']
interpolated_MagSensor_2 = Camera_Synced_Sensors['interpolated_MagSensor_2']
interpolated_Right_Arm = Camera_Synced_Sensors['interpolated_Right_Arm']
interpolated_Left_Arm = Camera_Synced_Sensors['interpolated_Left_Arm']
interpolated_Putter_Sensor = Camera_Synced_Sensors['interpolated_Putter_Sensor']



# Assuming the first column is the time axis for all sensors
time_axis = interpolated_MagSensor_1.iloc[:, 0]

# Create the figure with shared x-axes
fig = make_subplots(rows=5, cols=1, shared_xaxes=True)

# Helper function to add traces for cleanliness
def add_trace(fig, df, row, sensor_name):
    data_series = df.iloc[:, 3]  # Assuming the second column is what you want to plot
    fig.add_trace(go.Scatter(x=time_axis, y=data_series, mode='lines', name=sensor_name), row=row, col=1)

# Add traces for each sensor
add_trace(fig, interpolated_MagSensor_1, 1, 'Interpolated MagSensor 1')
add_trace(fig, interpolated_MagSensor_2, 2, 'Interpolated MagSensor 2')
add_trace(fig, interpolated_Right_Arm, 3, 'Interpolated Right Arm')
add_trace(fig, interpolated_Left_Arm, 4, 'Interpolated Left Arm')
add_trace(fig, interpolated_Putter_Sensor, 5, 'Interpolated Putter Sensor')


# Update layout and set the title
fig.update_layout(height=1200, width=1000, title_text="Comparison of Interpolated Sensors Time Series")
fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="Sensor Data")

# Show the figure
fig.show()


In [ ]:
#plot full kin/emg and markers of camera start stop 
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Access the DataFrames directly from the Sensor_Dict dictionary
interpolated_MagSensor_1 = Sensor_Dict['interpolated_MagSensor_1']
interpolated_MagSensor_2 = Sensor_Dict['interpolated_MagSensor_2']
interpolated_Right_Arm = Sensor_Dict['interpolated_Right_Arm']
interpolated_Left_Arm = Sensor_Dict['interpolated_Left_Arm']
interpolated_Putter_Sensor = Sensor_Dict['interpolated_Putter_Sensor']

# Assuming the first column is the time axis for all sensors
time_axis = interpolated_MagSensor_1.iloc[:, 0]

# Create the figure with shared x-axes
fig = make_subplots(rows=5, cols=1, shared_xaxes=True, subplot_titles=('Interpolated MagSensor 1', 'Interpolated MagSensor 2', 'Interpolated Right Arm', 'Interpolated Left Arm', 'Interpolated Putter Sensor'))

# Helper function to add traces for cleanliness
def add_trace(fig, df, row, sensor_name):
    data_series = df.iloc[:, 1]  # Assuming the second column is what you want to plot
    fig.add_trace(go.Scatter(x=time_axis, y=data_series, mode='lines', name=sensor_name), row=row, col=1)

# Add traces for each sensor
add_trace(fig, interpolated_MagSensor_1, 1, 'Interpolated MagSensor 1')
add_trace(fig, interpolated_MagSensor_2, 2, 'Interpolated MagSensor 2')
add_trace(fig, interpolated_Right_Arm, 3, 'Interpolated Right Arm')
add_trace(fig, interpolated_Left_Arm, 4, 'Interpolated Left Arm')
add_trace(fig, interpolated_Putter_Sensor, 5, 'Interpolated Putter Sensor')

# Assuming aligned_data is your ndarray and the first column contains the indices
first_index = aligned_data[0, 0]  # First observation index in aligned_data
last_index = aligned_data[-1, 0]  # Last observation index in aligned_data

# Get the time values for the first and last observation
first_time = time_axis.iloc[first_index]
last_time = time_axis.iloc[last_index]

# Function to add vertical lines
def add_vertical_line(fig, x, row, color='red'):
    fig.add_shape(type='line', 
                  x0=x, y0=0, x1=x, y1=1, 
                  xref='x'+str(row), yref='paper',
                  line=dict(color=color, width=2),
                  row=row, col=1)

# Add vertical lines for the first and last observation across all subplots
for i in range(1, 6):  # Adjust the range according to the number of subplots
    add_vertical_line(fig, first_time, i)
    add_vertical_line(fig, last_time, i)

# Update layout and set the title
fig.update_layout(height=1500, width=1000, title_text="Comparison of Interpolated Sensors Time Series")
fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="Sensor Data")

# Show the figure
fig.show()


In [ ]:
# List of variable names to keep 
variables_to_keep = ['Camera_Synced_Sensors', 'aligned_data', 'Sensor_Dict', 'file_paths', 'video_paths', 'write_paths', 'save_paths', 'participant', 'condition', 'hand_type','aligned_data_save']
# Iterate over the list of variables in the global namespace
for variable in list(globals()):
    # If the variable is not in the list to keep, delete it
    if variable not in variables_to_keep:
        del globals()[variable]
# Now, only the variables in 'variables_to_keep' should remain
del variable


In [ ]:
#### FOR MAC ############## Engine to input data for perfect syncing video to EMG###############

import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from IPython.display import display, clear_output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.graph_objs import FigureWidget
import ipywidgets as widgets
from ipywidgets import IntText

# Data loading section
# Load your data
time_series_data_1 = Camera_Synced_Sensors.get('interpolated_MagSensor_1')['local_flux'].values
time_series_data_2 = Camera_Synced_Sensors.get('interpolated_Putter_Sensor')['m/(s^2).1'].values

# Define the paths to your video files using the video_paths dictionary
video_file_path = video_paths["Videopanel_Sync"]  # Ensure the key matches exactly what you defined above

# Initialize a global list to store click coordinates or frame indices
clicked_points = []
# Initialize a list to store the values
offset_window_info = []

# Video capture setup
cap = cv2.VideoCapture(video_file_path)
window_width = 15000
start_frame = 1
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize the input widget for offset value but do not display it yet
offset_input = IntText(
    value=0,
    description='Set Offset:',
    disabled=False
)

# Right after defining offset_input
confirmed_offset = 0

# Define the function that updates confirmed_offset based on the widget's input
def update_confirmed_offset(change):
    global confirmed_offset
    confirmed_offset = change['new'] 
    print("Observer triggered, confirmed_offset updated to:", confirmed_offset)

# Attach the observer to the offset_input widget
offset_input.observe(update_confirmed_offset, names='value')

pause = False

# Function to convert matplotlib figure to an OpenCV image
def plot_to_image(figure):
    """Convert a Matplotlib figure to an OpenCV image."""
    canvas = FigureCanvas(figure)
    canvas.draw()
    img = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
    img = img.reshape(figure.canvas.get_width_height()[::-1] + (3,))
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
def display_interactive_graph(window_data_1, window_data_2, current_frame, total_frames, y_min_1, y_max_1, y_min_2, y_max_2):
    clear_output(wait=True)  # Clear the previous output, including Plotly graphs 
    # Create an X-axis that spans the entire range of the video/data
    full_x_values = list(range(window_start, window_end))  # Adjust to show the correct range
    
    # Creating subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        subplot_titles=("Sensor 1 Data", "Sensor 2 Data"))
    
    # Adding traces to the subplots
    fig.add_trace(go.Scatter(x=full_x_values, y=window_data_1, mode='lines+markers', name='Sensor 1'),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=full_x_values, y=window_data_2, mode='lines+markers', name='Sensor 2', line=dict(color='red')),
                  row=2, col=1)
    
    # Adjust the view to zoom in on the current window around the pause point
    fig.update_layout(
        title_text="Sensor Data vs. Video Frame",
        xaxis_title="Video Frame Index",
        yaxis1=dict(range=[y_min_1, y_max_1]),
        yaxis2=dict(range=[y_min_2, y_max_2]),
        xaxis2=dict(title="Video Frame Index", range=[current_frame - window_width // 2, current_frame + window_width // 2])
    )
    
    # Add a vertical red line at the current frame (midpoint) for both subplots
    for i in range(2):
        fig.add_shape(
            # Line Vertical
            dict(
                type="line",
                x0=current_frame,
                y0=min(y_min_1, y_min_2),
                x1=current_frame,
                y1=max(y_max_1, y_max_2),
                line=dict(
                    color="Red",
                    width=3
                )
            ),
            row=i+1, col=1
        )

    # Event handler for capturing clicks on the graph
    def on_click(trace, points, selector):
        for point in points.point_inds:
            clicked_index = full_x_values[point]  # Get the x-value (frame index) of the clicked point
            clicked_points.append(clicked_index)  # Save the clicked frame index
            print(f"Clicked on index: {clicked_index}")  # Optional: Print or process the clicked index as needed

    # Attach the click event handler to the figure
    scatter = fig.data[0]
    scatter.on_click(on_click)

    display(fig)  # Use IPython's display to show the fig

plt.ion()
fig, ax = plt.subplots(2, 1, figsize=(10, 8))

while True:
    if not pause:
        ret, frame = cap.read()
        if not ret:
            break
        current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1

    # Use confirmed_offset in the calculation
    window_start = max(0, current_frame - window_width // 2 + confirmed_offset)
    window_end = min(len(time_series_data_1), current_frame + window_width // 2 + 1 + confirmed_offset)
    window_data_1 = time_series_data_1[window_start:window_end]
    window_start_2 = max(0, current_frame - window_width // 2 + confirmed_offset)
    window_end_2 = min(len(time_series_data_2), current_frame + window_width // 2 + 1 + confirmed_offset)
    window_data_2 = time_series_data_2[window_start_2:window_end_2]
    

    if pause:
        key = cv2.waitKey(1) & 0xFF
        if key == ord('p'):
            pause = False
        elif key == ord('q'):
            break
        else:
            continue

    # Before your loop or inside it before plotting, calculate the middle index relative to the window_data array
    middle_index = len(window_data_1) // 2  
    ax[0].clear()
    ax[0].plot(window_data_1)
    ax[0].axvline(x=middle_index, color='r', linestyle='--')  # Adds a vertical red line at the middle of the plot
    ax[0].set_title('Sensor 1 Data')
    ax[1].clear()
    ax[1].plot(window_data_2, color='red')
    ax[1].axvline(x=middle_index, color='r', linestyle='--')  # Adds a vertical red line at the middle of the plot
    ax[1].set_title('Sensor 2 Data')
    fig.canvas.draw()
    plot_image = plot_to_image(fig)

    # Resize plot image to fit on video frame and adjust position to top left corner
    plot_image_resized = cv2.resize(plot_image, (int(frame.shape[1] * 0.25), int(frame.shape[0] * 0.5)))
    x_offset = 0
    y_offset = 0
    frame[y_offset:y_offset+plot_image_resized.shape[0], x_offset:x_offset+plot_image_resized.shape[1]] = plot_image_resized

    cv2.imshow('Frame with Time Series', frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('p'):
        pause = not pause  # Toggle the pause state
        if pause:
            # Logic when pausing
            y_min_1 = min(window_data_1)
            y_max_1 = max(window_data_1)
            y_min_2 = min(window_data_2)
            y_max_2 = max(window_data_2)

            # Display the Plotly graph for consulting
            display_interactive_graph(window_data_1, window_data_2, current_frame, total_frames, y_min_1, y_max_1, y_min_2, y_max_2)

            # Then, display the manual input widget for the offset value
            display(offset_input)
        else:
            # Logic when resuming from pause
            # Update confirmed_offset with the value entered during pause
            confirmed_offset = offset_input.value
            # Here, you might need to apply the confirmed_offset in adjusting your data display logic

    elif key == ord('q'):
        break

# Cleanup remains unchanged
cap.release()
cv2.waitKey(1)  # Sometimes helps with window closing
cv2.destroyAllWindows()
cv2.destroyWindow('Frame with Time Series')
plt.close(fig)

 


In [ ]:
# #FOR PC ############# Engine to input data for perfect syncing video to EMG###############
# %matplotlib widget
# import cv2
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
# from IPython.display import display, clear_output
# import plotly.graph_objects as go
# from plotly.graph_objs import FigureWidget
# import ipywidgets as widgets
# from ipywidgets import IntText  # Import IntText explicitly


# # Data loading section
# # Load your data
# time_series_data = Camera_Synced_Sensors.get('interpolated_MagSensor_1')['local_flux'].values
# # time_series_data = Camera_Synced_Sensors.get('interpolated_Putter_Sensor')['m/(s^2).1'].values

# # # Video file path setup
# # video_path = "/Users/beorn/Dropbox/0WORK/3Project Wensen/EMG Kin Research Experiment/Data/Experiment Data/SDC_007/Videos/Videopanel/SDC_007_Ball_RH_VP_synced_time_injection.MP4"

# # Define the paths to your video files using the video_paths dictionary
# video_file_path = video_paths["Videopanel_Sync"]  # Ensure the key matches exactly what you defined above


# # Initialize a global list to store click coordinates or frame indices
# clicked_points = []
# # Initialize a list to store the values
# offset_window_info = []

# # Video capture setup
# cap = cv2.VideoCapture(video_file_path)
# window_width = 5000
# start_frame = 200
# cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
# total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# # Initialize the input widget for offset value but do not display it yet
# offset_input = IntText(
#     value=0,
#     description='Set Offset:',
#     disabled=False
# )

# # Right after defining offset_input
# confirmed_offset = 0


# # Define the function that updates confirmed_offset based on the widget's input
# def update_confirmed_offset(change):
#     global confirmed_offset
#     confirmed_offset = change['new'] 
#     print("Observer triggered, confirmed_offset updated to:", confirmed_offset)
 

# # Attach the observer to the offset_input widget
# offset_input.observe(update_confirmed_offset, names='value')

# pause = False

# # Function to convert matplotlib figure to an OpenCV image
# def plot_to_image(figure):
#     """Convert a Matplotlib figure to an OpenCV image."""
#     canvas = FigureCanvas(figure)
#     canvas.draw()
#     img = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
#     img = img.reshape(figure.canvas.get_width_height()[::-1] + (3,))
#     return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    

# def display_interactive_graph(window_data, current_frame, total_frames, y_min, y_max):
#     clear_output(wait=True)  # Clear the previous output, including Plotly graphs 
#     # Create an X-axis that spans the entire range of the video/data
#     full_x_values = list(range(window_start, window_end))  # Adjust to show the correct range
#     # Creating a FigureWidget for interactive plotting
#     fig = FigureWidget([go.Scatter(x=full_x_values, y=window_data, mode='lines+markers')])
#     # Adjust the view to zoom in on the current window around the pause point
#     fig.update_layout(
#         title_text="Sensor Data vs. Video Frame",
#         xaxis_title="Video Frame Index",
#         yaxis=dict(range=[y_min, y_max]),
#         xaxis_range=[current_frame - window_width // 2, current_frame + window_width // 2]
#     )
#         # Add a vertical red line at the current frame (midpoint)
#     fig.add_shape(
#         # Line Vertical
#         dict(
#             type="line",
#             x0=current_frame,
#             y0=y_min,
#             x1=current_frame,
#             y1=y_max,
#             line=dict(
#                 color="Red",
#                 width=3
#             )
#     ))

#     # Event handler for capturing clicks on the graph
#     def on_click(trace, points, selector):
#         for point in points.point_inds:
#             clicked_index = full_x_values[point]  # Get the x-value (frame index) of the clicked point
#             clicked_points.append(clicked_index)  # Save the clicked frame index
#             print(f"Clicked on index: {clicked_index}")  # Optional: Print or process the clicked index as needed
#     # Attach the click event handler to the figure
#     scatter = fig.data[0]
#     scatter.on_click(on_click)

#     display(fig)  # Use IPython's display to show the fig
    

# # Note: You might need to adjust the range calculation for `full_x_values` based on your specific needs.

# # perhaps needed for something we don't need it for anymore
# plt.ion()
# fig, ax = plt.subplots()
# # cid = fig.canvas.mpl_connect('button_press_event', on_click)


# desired_width = 1280  # Change this to your desired width
# desired_height = 720  # Change this to your desired height

# while True:
#     if not pause:
#         ret, frame = cap.read()
#         if not ret:
#             break
#         # Resize the video frame to the desired dimensions
#         frame = cv2.resize(frame, (desired_width, desired_height))
#         current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1

#     # Use confirmed_offset in the calculation
#     window_start = max(0, current_frame - window_width // 2 + confirmed_offset)
#     window_end = min(len(time_series_data), current_frame + window_width // 2 + 1 + confirmed_offset)
#     window_data = time_series_data[window_start:window_end]

#     if pause:
#         key = cv2.waitKey(1) & 0xFF
#         if key == ord('p'):
#             pause = False
#         elif key == ord('q'):
#             break
#         else:
#             continue

#     middle_index = len(window_data) // 2  
#     ax.clear()
#     ax.plot(window_data)
#     ax.axvline(x=middle_index, color='r', linestyle='--')  # Adds a vertical red line at the middle of the plot
#     fig.canvas.draw()
#     plot_image = plot_to_image(fig)

#     # Resize plot image to fit on video frame and adjust position to top left corner
#     plot_image_resized = cv2.resize(plot_image, (int(desired_width * 0.25), int(desired_height * 0.5)))
#     x_offset = 0
#     y_offset = 0
#     frame[y_offset:y_offset+plot_image_resized.shape[0], x_offset:x_offset+plot_image_resized.shape[1]] = plot_image_resized

#     cv2.imshow('Frame with Time Series', frame)

#     key = cv2.waitKey(1) & 0xFF
#     if key == ord('p'):
#         pause = not pause
#         if pause:
#             y_min = min(window_data)
#             y_max = max(window_data)
#             display_interactive_graph(window_data, current_frame, total_frames, y_min, y_max)
#             display(offset_input)
#         else:
#             confirmed_offset = offset_input.value
#     elif key == ord('q'):
#         break

# cap.release()
# cv2.waitKey(1)  # Sometimes helps with window closing
# cv2.destroyAllWindows()  # This destroys all windows
# plt.close(fig)  # Close the matplotlib plot if it's no longer needed



In [ ]:
#calculate Frame Correction and Starting Frame for video sync

frame_correction = confirmed_offset-current_frame


confirmed_start_frame = current_frame




In [ ]:
# direct .json save
import numpy as np
import json
import os

# Load existing data from the JSON file
def load_existing_data(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            return json.load(file)
    else:
        return {
            "Ball": {"BH": {"data": [], "offset": None},
                     "RH": {"data": [], "offset": None},
                     "LH": {"data": [], "offset": None}},
            "NB": {"BH": {"data": [], "offset": None},
                   "RH": {"data": [], "offset": None},
                   "LH": {"data": [], "offset": None}}
        }

# Define the file path for the JSON file
output_directory = os.path.dirname(aligned_data_save["aligned_data_path"])
output_file = os.path.basename(aligned_data_save["aligned_data_path"])
output_path = os.path.join(output_directory, output_file)

# Make sure the directory exists
os.makedirs(output_directory, exist_ok=True)

# Load existing data or initialize if not present
data_to_save = load_existing_data(output_path)

# Insert the aligned_data into the correct condition and hand type, converting ndarray to list
data_to_save[condition][hand_type]['data'] = aligned_data.tolist()
data_to_save[condition][hand_type]['offset'] = frame_correction  # Store the offset

# Write the updated data to the JSON file
with open(output_path, 'w') as json_file:
    json.dump(data_to_save, json_file, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
#Saves Time axis To Matlab deployable file in the appropriate directory this one works

import numpy as np
import scipy.io
import os

# Define the path to save the .mat file
save_save_path = save_paths["Save_Sync"]

# Load the existing data from the .mat file if it exists, otherwise initialize it
try:
    loaded_data = scipy.io.loadmat(save_save_path, squeeze_me=True)
    if 'Time_Axis_Alignment' in loaded_data:
        time_axis_alignment = loaded_data['Time_Axis_Alignment']
        existing_data = {}
        for condition_key in ['Ball', 'NB']:
            if condition_key in time_axis_alignment.dtype.names:
                existing_data[condition_key] = {}
                condition_data = time_axis_alignment[condition_key].item()
                for hand_key in ['BH', 'RH', 'LH']:
                    if hand_key in condition_data.dtype.names:
                        existing_data[condition_key][hand_key] = condition_data[hand_key].item()
                    else:
                        existing_data[condition_key][hand_key] = ['no data', 'no data']
            else:
                existing_data[condition_key] = {
                    'BH': ['no data', 'no data'],
                    'RH': ['no data', 'no data'],
                    'LH': ['no data', 'no data']
                }
    else:
        raise KeyError("Time_Axis_Alignment not found in loaded data.")
except (FileNotFoundError, KeyError):
    existing_data = {
        'Ball': {
            'BH': ['no data', 'no data'],
            'RH': ['no data', 'no data'],
            'LH': ['no data', 'no data']
        },
        'NB': {
            'BH': ['no data', 'no data'],
            'RH': ['no data', 'no data'],
            'LH': ['no data', 'no data']
        }
    }

# Ask the user which hand configuration to update
#choice = input("Choose the hand configuration to update (BH, RH, LH): ").strip().upper()
# Use the global variable for hand configuration
choice = hand_type

# Prepare the data to be saved
aligned_data_double = aligned_data.astype(np.float64)
frame_correction_double = np.array([frame_correction], dtype=np.float64)

# Update only the chosen hand configuration and condition in the existing data
if choice in existing_data[condition]:
    if choice == 'RH':
        existing_data[condition][choice] = [[aligned_data_double, frame_correction_double]]
    else:
        existing_data[condition][choice] = [aligned_data_double, frame_correction_double]

# Ensure the save directory exists
os.makedirs(os.path.dirname(save_save_path), exist_ok=True)

# Prepare the nested structure for saving
time_axis_alignment_struct = {
    condition_key: {
        hand_key: np.array(existing_data[condition_key][hand_key], dtype=object)
        for hand_key in existing_data[condition_key]
    } for condition_key in existing_data
}

# Save the updated data to a .mat file
scipy.io.savemat(save_save_path, {
    'Time_Axis_Alignment': time_axis_alignment_struct
})

print(f"Data saved for {choice} under condition {condition} in the 'Time Axis Alignment' structure.")


In [ ]:
# #### Test accuracy of syncing with Video playback, record video for posterity 

# import cv2
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
# from IPython.display import display, clear_output

# # Load your time series data
# time_series_data1 = Camera_Synced_Sensors.get('interpolated_MagSensor_1')['local_flux'].values
# time_series_data2 = Camera_Synced_Sensors.get('interpolated_MagSensor_2')['local_flux'].values
# time_series_data3 = Camera_Synced_Sensors.get('interpolated_Right_Arm')['m/(s^2)'].values
# time_series_data4 = Camera_Synced_Sensors.get('interpolated_Right_Arm')['m/(s^2).1'].values
# time_series_data5 = Camera_Synced_Sensors.get('interpolated_Right_Arm')['m/(s^2).2'].values
# time_series_data6 = Camera_Synced_Sensors.get('interpolated_Putter_Sensor')['m/(s^2).2'].values
# time_series_data7 = Camera_Synced_Sensors.get('interpolated_Supinator_Flex_Carp_Uln_R')['mV'].values
# time_series_data8 = Camera_Synced_Sensors.get('interpolated_Pron_Teres_pron_quad_R')['mV'].values
# time_series_data9 = Camera_Synced_Sensors.get('interpolated_Pron_Teres_pron_quad_R')['mV.1'].values
# time_series_data10 = Camera_Synced_Sensors.get('interpolated_Supinator_Flex_Carp_Uln_L')['mV'].values

# # Path to your video file
# video_file_path = video_paths["Videopanel_Sync"]  # Ensure this key matches your variable definition
# write_file_path = write_paths["Write_Sync"]  # Ensure this key matches your variable definition

# # Window settings
# window_width = 200  # Width of the time-series window to display
# start_frame = confirmed_start_frame - 50  # Adjust start frame to capture earlier data

# # Initialize OpenCV VideoCapture
# cap = cv2.VideoCapture(video_file_path)
# cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
# current_frame = start_frame

# # Function to convert matplotlib figure to OpenCV image
# def plot_to_image(figure):
#     canvas = FigureCanvas(figure)
#     canvas.draw()
#     img = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
#     img = img.reshape(figure.canvas.get_width_height()[::-1] + (3,))
#     return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

# # Setting up the video writer with the correct codec
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for MP4 files
# out_frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# out_frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# out_video = cv2.VideoWriter(write_file_path, fourcc, 30.0, (out_frame_width, out_frame_height))

# # Interactive plot setup
# plt.ion()
# fig, axs = plt.subplots(10, 1, figsize=(5, 7))  # Adjust subplots if needed

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     current_frame += 1

#     # For each time series, calculate and plot the window
#     for i, time_series_data in enumerate([
#         time_series_data1, time_series_data2, time_series_data3, time_series_data4,
#         time_series_data5, time_series_data6, time_series_data7, time_series_data8,
#         time_series_data9, time_series_data10
#     ]):
#         window_start = max(0, current_frame - window_width // 2 + frame_correction)
#         window_end = min(len(time_series_data), current_frame + window_width // 2 + 1 + frame_correction)
#         window_data = time_series_data[window_start:window_end]

#         # Plot each subplot
#         axs[i].clear()
#         axs[i].plot(window_data)
#         middle_index = len(window_data) // 2
#         axs[i].axvline(x=middle_index, color='r', linestyle='--')  # Middle red vertical line
    
#     fig.canvas.draw()
#     plot_image = plot_to_image(fig)

#     # Resize and overlay the plot image onto the video frame
#     plot_image_resized = cv2.resize(plot_image, (int(frame.shape[1] * 0.4), int(frame.shape[0] * 0.8)))
#     x_offset, y_offset = 0, 0
#     frame[y_offset:y_offset+plot_image_resized.shape[0], x_offset:x_offset+plot_image_resized.shape[1]] = plot_image_resized

#     cv2.imshow('Frame with Time Series', frame)
#     out_video.write(frame)  # Save the frame to the output video

#     # Quit on 'q' key press
#     key = cv2.waitKey(1) & 0xFF
#     if key == ord('q'):
#         break

# # Cleanup resources
# cap.release()
# out_video.release()
# cv2.destroyAllWindows()
# plt.close(fig)
